https://www.pyimagesearch.com/2019/03/04/holistically-nested-edge-detection-with-opencv-and-deep-learning/

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import glob
import os

import cv2
from keras.preprocessing.image import load_img, save_img
from keras.utils import get_file
import numpy as np

import config

In [ ]:
# change current directory to project root
current_dir = os.getcwd().split('/')[-1]
if current_dir != 'pix2pix-edges-with-color': 
    %cd '..'

### Download Zappos50K dataset

In [ ]:
get_file(f'{config.ZAPPOS_DATASET_NAME}.zip', config.ZAPPOS_DATASET_URL, extract=True, 
         cache_dir='.', cache_subdir='')

In [ ]:
def detect_edges(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    img_gray = cv2.bilateralFilter(img_gray, 5, 50, 50)
    img_gray_edges = cv2.Canny(img_gray, 45, 100)
    img_gray_edges = cv2.bitwise_not(img_gray_edges) # invert black/white
    img_edges = cv2.cvtColor(img_gray_edges, cv2.COLOR_GRAY2RGB)
    
    return img_edges

def get_shoe_model_id_to_filepaths_dict(data_dir):
    """The naming convention for each image in the Zappos dataset is SHOE_MODEL_ID.SHOE_COLOR_PATTERN_ID.jpg.
    Only images with a minimum frequency of two shoe model ids  are kept. 

    e.g. For the list of image names below, image 2.1.jpg will not be kept since the shoe model id only occurs once.
    1.1.jpg
    1.2.jpg
    2.1.jpg
    """
    model_id_to_filepaths = {}
    for filepath in glob.glob(f'{data_dir}/*.jpg'):
        filename = os.path.basename(filepath)
        model_id, pattern_id = filename.split('.')[:2]
        filepaths = model_id_to_filepaths.get(model_id, [])
        filepaths.append(filepath)
        model_id_to_filepaths[model_id] = filepaths
    
    model_id_to_filepaths = {model_id: filepaths 
                             for model_id, filepaths in model_id_to_filepaths.items()
                             if len(filepaths) >= config.ZAPPOS_DATASET_MIN_SHOE_MODEL_ID_COUNT}
    
    return model_id_to_filepaths

###  Copy target images

In [ ]:
# data_dir = f'{config.ZAPPOS_DATASET_SNEAKERS_DIR}/Nike*'
# model_id_to_filepaths = get_shoe_model_id_to_filepaths_dict(data_dir)

# for filepaths in model_id_to_filepaths.values():
#     for filepath in filepaths:
#         filename = os.path.basename(filepath)
#         img_target = load_img(filepath, target_size=(config.IMG_HEIGHT, config.IMG_WIDTH))
#         img_target = np.array(img_target)
#         img_target_filepath = os.path.join(config.TRAINING_TARGET_DIR, filename)
#         save_img(img_target_filepath, img_target)

### Create source (edge) images

In [ ]:
model_id_to_filepaths = get_shoe_model_id_to_filepaths_dict(config.TRAINING_TARGET_DIR)

for filepaths in model_id_to_filepaths.values():
    img_first = load_img(filepaths[0], target_size=(config.IMG_HEIGHT, config.IMG_WIDTH))
    img_first = np.array(img_first)
    img_edges = detect_edges(img_first)
    
    for filepath in filepaths:
        filename = os.path.basename(filepath)
        img_source_filepath = os.path.join(config.TRAINING_SOURCE_DIR, filename)
        save_img(img_source_filepath, img_edges)